In [19]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

!pip install pyspark

Defaulting to user installation because normal site-packages is not writeable


<h3>1) Работа с PySpark</h3>

In [20]:
from pyspark.sql import SparkSession

# Создаем Spark-сессию
spark = SparkSession.builder \
    .appName("PySpark Example") \
    .master("local[*]") \
    .getOrCreate()

# Загружаем данные из локального CSV-файла
flights_pd = pd.read_csv("./data/flights.csv")

flights_df = spark.createDataFrame(flights_pd)

flights_df.show(5)


+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
| id|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|                name|
+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
|  0|2013|    1|  1|   517.0|           515|      2.0|   830.0|           819|     11.0|     UA|  1545| N14228|   EWR| IAH|   227.0|    1400|   5|    15|2013-01-01 05:00:00|United Air Lines ...|
|  1|2013|    1|  1|   533.0|           529|      4.0|   850.0|           830|     20.0|     UA|  1714| N24211|   LGA| IAH|   227.0|    1416|   5|    29|2013-01-01 05:00:00|United Air Lines ...|
|  2|2013|    1|  1|   54

In [21]:
# Пример запроса: выбор строк, где задержка больше 60 минут
flights_df = flights_df.filter(flights_df['arr_delay'] > 60)

# Показать результаты
flights_df.show(5)

+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
| id|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|                name|
+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
|119|2013|    1|  1|   811.0|           630|    101.0|  1047.0|           830|    137.0|     MQ|  4576| N531MQ|   LGA| CLT|   118.0|     544|   6|    30|2013-01-01 06:00:00|           Envoy Air|
|151|2013|    1|  1|   848.0|          1835|    853.0|  1001.0|          1950|    851.0|     MQ|  3944| N942MQ|   JFK| BWI|    41.0|     184|  18|    35|2013-01-01 18:00:00|           Envoy Air|
|218|2013|    1|  1|   95

In [22]:
!pip install hdfs


Defaulting to user installation because normal site-packages is not writeable


<h3>2) Работа с HDSM</h3>

In [32]:
!docker pull sequenceiq/hadoop-docker:2.7.3
!docker run -it --rm -p 9870:9870 -p 8088:8088 sequenceiq/hadoop-docker:2.7.3


Error response from daemon: failed to resolve reference "docker.io/sequenceiq/hadoop-docker:2.7.3": docker.io/sequenceiq/hadoop-docker:2.7.3: not found
the input device is not a TTY.  If you are using mintty, try prefixing the command with 'winpty'


In [30]:
from hdfs import InsecureClient

# Убедитесь, что Hadoop запущен и доступен по указанному адресу и порту
client = InsecureClient('http://localhost:9870', user='hdfs')

# Проверьте подключение
try:
    status = client.status('/')
    print("Successfully connected to HDFS!")
    print(status)
except Exception as e:
    print(f"Failed to connect to HDFS: {e}")


Failed to connect to HDFS: HTTPConnectionPool(host='localhost', port=9870): Max retries exceeded with url: /webhdfs/v1/?user.name=hdfs&op=GETFILESTATUS (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002C59E67E8B0>: Failed to establish a new connection: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение'))


In [31]:
# Создаем текстовый файл с данными
data = "Hello, Hadoop!\nThis is a test file."

# Записываем данные в локальный файл
with open('test_file.txt', 'w') as f:
    f.write(data)

# Загружаем файл в HDFS
client.write('/user/hdfs/test_file.txt', data=data, encoding='utf-8', overwrite=True)


ConnectionError: HTTPConnectionPool(host='localhost', port=9870): Max retries exceeded with url: /webhdfs/v1/user/hdfs/test_file.txt?user.name=hdfs&overwrite=True&op=CREATE (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002C59862FD10>: Failed to establish a new connection: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение'))

<h3>3) Работа с MongoDB</h3>

In [25]:
!pip install pymongo mongomock


Defaulting to user installation because normal site-packages is not writeable


In [26]:
from pymongo import MongoClient

# Подключение к MongoDB
client = MongoClient('localhost', 27017)

# Получение доступа к базе данных
db = client['test_database']

# Получение доступа к коллекции
collection = db['test_collection']


In [27]:
from mongomock import MongoClient

# Создание мок-клиента MongoDB
mock_client = MongoClient()

# Получение доступа к мок-базе данных
mock_db = mock_client['test_database']

# Получение доступа к мок-коллекции
mock_collection = mock_db['test_collection']

# Пример данных
data = [
    {"name": "setosa", "petal_length": 1.4, "petal_width": 0.2},
    {"name": "versicolor", "petal_length": 4.5, "petal_width": 1.3},
    {"name": "virginica", "petal_length": 6.3, "petal_width": 2.5}
]

# Вставка данных в мок-коллекцию
mock_collection.insert_many(data)

# Чтение данных из мок-коллекции
results = mock_collection.find()
for document in results:
    print(document)


{'name': 'setosa', 'petal_length': 1.4, 'petal_width': 0.2, '_id': ObjectId('67e4d08608220861ffeae88d')}
{'name': 'versicolor', 'petal_length': 4.5, 'petal_width': 1.3, '_id': ObjectId('67e4d08608220861ffeae88e')}
{'name': 'virginica', 'petal_length': 6.3, 'petal_width': 2.5, '_id': ObjectId('67e4d08608220861ffeae88f')}


In [28]:
spark.stop()